# Multi-agent Customer Support Automation

In this lesson, you’ll explore how to build an **automated customer support system** using multiple agents that work together. You’ll also learn about the **six key elements** that make agents more effective:

1. **Role Playing** – Giving each agent a specific role (e.g., Support Representative, Quality Checker) so they act like real team members.  
2. **Focus** – Defining each agent’s goal clearly so they stay on track while solving customer issues.  
3. **Tools** – Equipping agents with the right resources (like knowledge bases, APIs, or documents) to answer questions accurately.  
4. **Cooperation** – Allowing agents to collaborate and share information to provide better solutions.  
5. **Guardrails** – Setting rules and limits so agents give safe, accurate, and professional responses.  
6. **Memory** – Enabling agents to remember past interactions, so support feels consistent and personalized.  

> Together, these elements create a **smarter, friendlier, and more reliable customer support automation system**.


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Get the API key and model from .env
groq_api_key = os.getenv("GROQ_API_KEY")
groq_model = os.getenv("GROQ_MODEL")

# Set environment variables for use in code
os.environ["GROQ_API_KEY"] = groq_api_key
os.environ["GROQ_MODEL"] = groq_model

print("Using Groq model:", groq_model)


Using Groq model: qwen/qwen3-32b


In [3]:
# Import LLM after setting environment variables
from crewai.llm import LLM

# Create the LLM instance with reduced max_tokens to avoid 8000 limit
llm = LLM(
    model=f"groq/{groq_model}",
    api_key=groq_api_key,
    temperature=0.7,
    timeout=60,
    max_retries=3
)

In [4]:
from crewai import Agent, Task, Crew

In [5]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
    llm=llm 
)

In [6]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    llm=llm
)

## Tools, Guardrails and Memory

# Agent and Task Level Tools in Agents

### 🔹 SerperDevTool → Search the web  
### 🔹 ScrapeWebsiteTool → Extract content from a page  
### 🔹 WebsiteSearchTool → Search inside a specific website  

---

## 1. Agent-Level Tools  

- Tools are attached **directly to the agent**.  
- The agent can use them for **any task** it performs.  
- These act like the agent’s **default toolbox**.  

👉 **Example:**

```python
researcher = Agent(
    role="Research Analyst",
    goal="Find AI trends",
    backstory="Expert at analyzing AI news",
    tools=[SerperDevTool(), ScrapeWebsiteTool()]  # agent-level tools
)
```
## 2. Task-Level Tools  

- Tools are attached to a **specific task**.  
- The agent will **only use them during that task**.  
- **Task tools override agent tools** → if task-level tools are given, the agent will only use those.  

👉 **Example:**

```python
task = Task(
    description="Scrape Hugging Face blog for AI updates",
    agent=researcher,
    tools=[ScrapeWebsiteTool()]  # task-level tool
)
```

# Tools

In [7]:
from crewai_tools import  ScrapeWebsiteTool


d:\Education\Machine Learning\Machine learning Project\CrewAI\.venv\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [8]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

## Creating a Task

In [9]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [10]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


## Creating the crew 

In [13]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  llm=llm
)


In [14]:
import time

# Add delay before execution to avoid rate limits
time.sleep(2)

# Execute with try-catch for better error handling
try:
    result = crew.kickoff(inputs={
        "customer": "AdilHayatAIDeveloper",
        "person": "Adil Hayat",
        "inquiry": (
            "I need help with setting up a Crew "
            "and kicking it off, specifically "
            "how can I add memory to my crew? "
            "Can you provide guidance?"
        )
    })
    print(result)

except Exception as e:
    if "token" in str(e).lower() or "rate" in str(e).lower():
        print("Rate limit or token limit reached. Waiting 30 seconds and retrying...")
        time.sleep(30)
        try:
            result = crew.kickoff(inputs={
                "customer": "AdilHayatAIDeveloper",
                "person": "Adil Hayat",
                "inquiry": (
                    "I need help with setting up a Crew "
                    "and kicking it off, specifically "
                    "how can I add memory to my crew? "
                    "Can you provide guidance?"
                )
            })
            print(result)
        except Exception as retry_error:
            print(f"Retry failed: {retry_error}")
    else:
        print(f"Error occurred: {e}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 59527f78-f745-4833-9407-0d54e75bf2ec                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: AdilHayatAIDeveloper just reached out with a super important ask:                                        │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Adil Hayat from AdilHayatAIDeveloper is the one that reached out. Make sure to use everything you know to      │
│  provide the best support possible.You must strive to provide a complete and accurate response to the           │
│  customer's inquiry.                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  To add memory to your Crew in CrewAI, you can enable it during the Crew initialization by setting the          │
│  `memory` parameter to `True`. Here's a detailed guide based on the official documentation:                     │
│                                                                                                                 │
│  1. **Basic Setup with Memory Enabled**:                                                                        │
│     When creating your `Crew` instance, include the `memory=True` argument. This allows the Crew to retain      │
│  information between task executions. Example:                                                                  │
│     ```python                                                                                                   │
│     from crewai import Crew, Process                                                                            │
│                                                                                                                 │
│     crew = Crew(                                                                                                │
│         agents=agents,  # Your defined agents                                                                   │
│         tasks=tasks,    # Your defined tasks                                                                    │
│         process=Process.sequential,  # or Process.hierarchical                                                  │
│         memory=True     # Enables basic memory functionality                                                    │
│     )                                                                                                           │
│     ```                                                                                                         │
│                                                                                                                 │
│  2. **Advanced Custom Memory (Optional)**:                                                                      │
│     For advanced use cases, you can implement a custom memory class by extending `CrewMemory` from              │
│  `crewai.memory`. This allows you to define how memory is stored, retrieved, and managed. Example:              │
│     ```python                                                                                                   │
│     from crewai.memory import CrewMemory                                                                        │
│                                                                                                                 │
│     class CustomCrewMemory(CrewMemory):                                                                         │
│         def save_memory(self, data):                                                                            │
│             # Custom save logic                                                                                 │
│             pass                                                                                                │
│                                                                                                                 │
│         def retrieve_memory(self, query):                                                                       │
│             # Custom retrieve logic                    

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ca97d536-74cb-4e16-856a-c4d876cb147f                                                                     │
│  Agent: Senior Support Representative                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Task: Review the response drafted by the Senior Support Representative for AdilHayatAIDeveloper's inquiry.     │
│  Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for      │
│  customer support.                                                                                              │
│  Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly    │
│  tone.                                                                                                          │
│  Check for references and sources used to  find the information, ensuring the response is well-supported and    │
│  leaves no questions unanswered.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Hey Adil! 😊 Thanks for reaching out with your question about adding memory to a Crew in CrewAI. Let’s break   │
│  it down step-by-step to ensure you’re all set to implement memory effectively.                                 │
│                                                                                                                 │
│  ### 🧠 **How to Enable Memory in Your Crew**                                                                   │
│  To enable memory, simply set `memory=True` when initializing your Crew. This activates the built-in key-value  │
│  memory system, which stores context between tasks. Here’s how to do it:                                        │
│                                                                                                                 │
│  ```python                                                                                                      │
│  from crewai import Crew, Process                                                                               │
│                                                                                                                 │
│  crew = Crew(                                                                                                   │
│      agents=agents,          # Your agents list                                                                 │
│      tasks=tasks,            # Your tasks list                                                                  │
│      process=Process.sequential,  # or Process.hierarchical                                                     │
│      memory=True             # ✅ Enables default memory system                                                 │
│  )                                                                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
│  This setup automatically retains data like intermediate outputs or decisions made by agents, allowing          │
│  seamless context-sharing across tasks. For example, if Agent A generates a report, Agent B can access it in    │
│  subsequent tasks without needing to reprocess inputs.                                                          │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 🛠️ **Advanced: Custom Memory Implementation**                                                               │
│  If you need more control (e.g., saving to a database or using vector storage), you can create a custom memory  │
│  class by subclassing `CrewMemory`. Here’s how to do it:                                                        │
│                                                                                                                 │
│  ```python                                                                                                      │
│  from crewai.memory import CrewMemory                    

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8e684ecb-9f34-4aed-be13-3c6e102dedec                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 59527f78-f745-4833-9407-0d54e75bf2ec                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Hey Adil! 😊 Thanks for reaching out with your question about adding memory to a Crew in         │
│  CrewAI. Let’s break it down step-by-step to ensure you’re all set to implement memory effectively.             │
│                                                                                                                 │
│  ### 🧠 **How to Enable Memory in Your Crew**                                                                   │
│  To enable memory, simply set `memory=True` when initializing your Crew. This activates the built-in key-value  │
│  memory system, which stores context between tasks. Here’s how to do it:                                        │
│                                                                                                                 │
│  ```python                                                                                                      │
│  from crewai import Crew, Process                                                                               │
│                                                                                                                 │
│  crew = Crew(                                                                                                   │
│      agents=agents,          # Your agents list                                                                 │
│      tasks=tasks,            # Your tasks list                                                                  │
│      process=Process.sequential,  # or Process.hierarchical                                                     │
│      memory=True             # ✅ Enables default memory system                                                 │
│  )                                                                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
│  This setup automatically retains data like intermediate outputs or decisions made by agents, allowing          │
│  seamless context-sharing across tasks. For example, if Agent A generates a report, Agent B can access it in    │
│  subsequent tasks without needing to reprocess inputs.                                                          │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 🛠️ **Advanced: Custom Memory Implementation**                                                               │
│  If you need more control (e.g., saving to a database or using vector storage), you can create a custom memory  │
│  class by subclassing `CrewMemory`. Here’s how to do it:                                                        │
│                                                                                                                 │
│  ```python                                              

Hey Adil! 😊 Thanks for reaching out with your question about adding memory to a Crew in CrewAI. Let’s break it down step-by-step to ensure you’re all set to implement memory effectively.  

### 🧠 **How to Enable Memory in Your Crew**  
To enable memory, simply set `memory=True` when initializing your Crew. This activates the built-in key-value memory system, which stores context between tasks. Here’s how to do it:  

```python
from crewai import Crew, Process

crew = Crew(
    agents=agents,          # Your agents list
    tasks=tasks,            # Your tasks list
    process=Process.sequential,  # or Process.hierarchical
    memory=True             # ✅ Enables default memory system
)
```  

This setup automatically retains data like intermediate outputs or decisions made by agents, allowing seamless context-sharing across tasks. For example, if Agent A generates a report, Agent B can access it in subsequent tasks without needing to reprocess inputs.  

---

### 🛠️ **Advanced: Custom M